In [2]:
import wandb

api = wandb.Api()

In [6]:
project_path = 'Fairness-wiki'
runs = api.runs(project_path)

for run in runs:
    group = run.group
    perplexity = run.summary.get('val/perplexity_mean')

    print(f"Run ID: {run.id}, Group: {group}, Perplexity: {perplexity}")


Run ID: 35m1hf8x, Group: fed-avg, Perplexity: 125.38703268257866
Run ID: izjotjf9, Group: local, Perplexity: 135.97526645931794
Run ID: 3uguy0g5, Group: val-sim, Perplexity: 121.67908272395896
Run ID: rnt9getz, Group: val-sim-w-penalizer, Perplexity: 118.87917227116688
Run ID: raet4p4h, Group: oracle-w-penalizer, Perplexity: 123.42238586984917
Run ID: a44zhppu, Group: oracle, Perplexity: 124.9359685497758
Run ID: wb1u9fh4, Group: fed-avg, Perplexity: 125.13610597927524
Run ID: ndr3qebu, Group: oracle-w-penalizer, Perplexity: 123.4019033603601
Run ID: q0wp64ll, Group: local, Perplexity: 139.41872038349513
Run ID: adpgk385, Group: val-sim, Perplexity: 118.14233918624667
Run ID: h4qt3j2z, Group: val-sim-w-penalizer, Perplexity: 115.10771583763956
Run ID: mdkca7m2, Group: oracle, Perplexity: 123.48319169939884
Run ID: ddxn5iar, Group: fed-avg, Perplexity: 123.93551102512646
Run ID: vug7vrxc, Group: oracle-w-penalizer, Perplexity: 122.94207065871176
Run ID: 9yqc0ozz, Group: local, Perplexit